In [1]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 689.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 13.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.0
    Uninstalling scipy-1.16.0:
      Successfully uninstalled scipy-1.16.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
open

In [1]:
import gensim

In [3]:
import gensim.downloader as api
wv=api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [4]:
wv.most_similar("computer")

[('computers', 0.7979379892349243),
 ('laptop', 0.6640493273735046),
 ('laptop_computer', 0.6548868417739868),
 ('Computer', 0.647333562374115),
 ('com_puter', 0.6082080006599426),
 ('technician_Leonard_Luchko', 0.5662748217582703),
 ('mainframes_minicomputers', 0.5617720484733582),
 ('laptop_computers', 0.5585449934005737),
 ('PC', 0.5539618730545044),
 ('maker_Dell_DELL.O', 0.5519254207611084)]

In [6]:
import pandas as pd
df = pd.read_csv("/content/Fake_Real_Data.csv", engine='python', sep=',', on_bad_lines='skip')
df.head()


,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [7]:
df["label_num"]=df["label"].map({"Fake":0,"Real":1})

In [8]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
# preprocess and vectorize the text
import spacy
nlp=spacy.load("en_core_web_lg")
def preprocess_and_vectorize(text):
  doc=nlp(text)
  filtered_token=[]
  for token in doc:
    if token.is_stop or token.is_punct:
      continue
    filtered_token.append(token.lemma_)
  return wv.get_mean_vector(filtered_token)


In [10]:
# getting new vectorize column
df["vector"]=df.Text.apply(lambda text: preprocess_and_vectorize(text))

In [11]:
# train test split
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(
    df.vector.values,
    df.label_num.values,
    test_size=0.2,
    random_state=2022,
    stratify=df.label_num)

In [12]:
import numpy as np

x_train_2d=np.stack(x_train) # for coverting to 2D
x_test_2d=np.stack(x_test)   # for coverting to 2D

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

rf=RandomForestClassifier()
rf.fit(x_train_2d,y_train)
y_pred=rf.predict(x_test_2d)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      0.97      0.97       159
           1       0.97      0.96      0.97       162

    accuracy                           0.97       321
   macro avg       0.97      0.97      0.97       321
weighted avg       0.97      0.97      0.97       321



In [20]:
rf.score(x_test_2d,y_test)

0.9688473520249221

In [15]:
gbc=GradientBoostingClassifier()
gbc.fit(x_train_2d,y_train)
y_pred=gbc.predict(x_test_2d)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       159
           1       0.98      0.98      0.98       162

    accuracy                           0.98       321
   macro avg       0.98      0.98      0.98       321
weighted avg       0.98      0.98      0.98       321



In [16]:
gbc.score(x_test_2d,y_test)

0.9813084112149533

In [18]:
# make some prediction
test_news="""President Donald Trump told CNN's Kaitlan Collins he wasn’t
aware of Ghislaine Maxwell’s prison transfer, and praised Deputy Attorney
General Todd Blanche when asked if he thought Ghislaine Maxwell is a credible witness."""

test_news_vector=[preprocess_and_vectorize(n) for n in [test_news]]
gbc.predict(test_news_vector)

array([0])

In [19]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test,y_pred))

[[156   3]
 [  3 159]]
